Здесь будут попытки разобраться с падежами (начиная с датива и аллатива)

In [201]:
import os, re, pickle
from pprint import pprint
import numpy as np
import pandas as pd

In [202]:
corpora = ['Kamchatka', 'Sebjan']

In [7]:
def make_wordlist(corp):
    wds = []
    for doc in corp:
        for i in range(len(corp[doc]['text'])):
            if 'mb' in corp[doc]['text'][i] and 'ps' in corp[doc]['text'][i]:
                wds += zip(corp[doc]['text'][i]['mb'], corp[doc]['text'][i]['ge'], corp[doc]['text'][i]['ps'])
    print(len(wds))
    return wds


for corp in corpora:
    with open('{}.pickle'.format(corp), 'rb') as f:
        content = pickle.load(f)
    wds = make_wordlist(content)
    with open('{}_wordlist.pickle'.format(corp), 'wb') as f:
        pickle.dump(wds, f)

33112
49800


In [192]:
from collections import Counter

def get_roots(affix, wdlist):
    res = []
    for wd in wdlist:
        if affix in wd[1]:
            res.append((wd[0][0], wd[1][0], wd[2][0]))
    return res

for corp in corpora:
    with open('../{}_wordlist.pickle'.format(corp), 'rb') as f:
        wdlist = pickle.load(f)
    datrts = Counter(get_roots('-dat', wdlist))
    keys = list(datrts.keys())
    datrts = pd.DataFrame({'morph': list(map(lambda x: x[0], keys)), 'gloss': list(map(lambda x: x[1], keys)), 
                           'pos': list(map(lambda x: x[2], keys)), 'count': list(map(lambda x: datrts[x], keys))})
    datrts = datrts.sort_values('count', ascending=False)
    datrts.to_pickle('{}_dative_roots.pickle'.format(corp))

In [22]:
s_dat = pd.read_pickle('Sebjan_dative_roots.pickle')
k_dat = pd.read_pickle('Kamchatka_dative_roots.pickle')
dat = pd.merge(s_dat, k_dat, how='outer', on=['morph', 'gloss', 'pos'], suffixes=['_s', '_k'])
dat.head()

,count_s,gloss,morph,pos,count_k
0,119.0,dist,tarak,pron,104.0
1,90.0,what,ịak,pron,51.0
2,76.0,summer,ireli,n,1.0
3,40.0,prox,erek,pron,30.0
4,32.0,year,anŋan,n,NaN


In [21]:
dat.to_excel('datives.xlsx')

In [30]:
for corp in corpora:
    with open('{}_wordlist.pickle'.format(corp), 'rb') as f:
        wdlist = pickle.load(f)
    allrts = Counter(get_roots('-all', wdlist))
    keys = list(allrts.keys())
    allrts = pd.DataFrame({'morph': list(map(lambda x: x[0], keys)), 'gloss': list(map(lambda x: x[1], keys)), 
                           'pos': list(map(lambda x: x[2], keys)), 'count': list(map(lambda x: allrts[x], keys))})
    allrts = allrts.sort_values('count', ascending=False)
    allrts.to_pickle('{}_allative_roots.pickle'.format(corp))

In [31]:
s_all = pd.read_pickle('Sebjan_allative_roots.pickle')
k_all = pd.read_pickle('Kamchatka_allative_roots.pickle')
allat = pd.merge(s_all, k_all, how='outer', on=['morph', 'gloss', 'pos'], suffixes=['_s', '_k'])
allat.head()

,count_s,gloss,morph,pos,count_k
0,36.0,dist,tarak,pron,25.0
1,32.0,prox,erek,pron,11.0
2,14.0,which,irek,pron,1.0
3,9.0,1sg.obl,min,pron,19.0
4,9.0,refl,meːn,pron,1.0


In [35]:
allat.loc[allat['pos']=='pron']

,count_s,gloss,morph,pos,count_k
0,36.0,dist,tarak,pron,25.0
1,32.0,prox,erek,pron,11.0
2,14.0,which,irek,pron,1.0
3,9.0,1sg.obl,min,pron,19.0
4,9.0,refl,meːn,pron,1.0
5,7.0,3sg,nọŋan,pron,8.0
10,4.0,1pl.in,mut,pron,1.0
12,4.0,what,ịak,pron,NaN
32,1.0,2pl.obl,hun,pron,1.0
35,1.0,all.R,vsё,pron,NaN


In [36]:
dat.loc[dat['pos']=='pron']

,count_s,gloss,morph,pos,count_k
0,119.0,dist,tarak,pron,104.0
1,90.0,what,ịak,pron,51.0
3,40.0,prox,erek,pron,30.0
5,31.0,1pl.in,mut,pron,NaN
7,29.0,1sg.obl,min,pron,24.0
11,21.0,3sg,nọŋan,pron,6.0
18,10.0,which,irek,pron,7.0
25,9.0,2sg.obl,hin,pron,4.0
52,4.0,who,ŋiː,pron,1.0
63,3.0,2pl.obl,hun,pron,3.0


In [37]:
cases = pd.merge(dat, allat, how='outer', on=['morph', 'gloss', 'pos'], suffixes=['_dat', '_all'])
cases.head()

,count_s_dat,gloss,morph,pos,count_k_dat,count_s_all,count_k_all
0,119.0,dist,tarak,pron,104.0,36.0,25.0
1,90.0,what,ịak,pron,51.0,4.0,NaN
2,76.0,summer,ireli,n,1.0,NaN,NaN
3,40.0,prox,erek,pron,30.0,32.0,11.0
4,32.0,year,anŋan,n,NaN,NaN,NaN


In [38]:
cases = cases[['morph', 'gloss', 'pos', 'count_s_dat', 'count_k_dat', 'count_s_all', 'count_k_all']]
cases.head()

,morph,gloss,pos,count_s_dat,count_k_dat,count_s_all,count_k_all
0,tarak,dist,pron,119.0,104.0,36.0,25.0
1,ịak,what,pron,90.0,51.0,4.0,NaN
2,ireli,summer,n,76.0,1.0,NaN,NaN
3,erek,prox,pron,40.0,30.0,32.0,11.0
4,anŋan,year,n,32.0,NaN,NaN,NaN


In [193]:
for corp in corpora:
    with open('../{}_wordlist.pickle'.format(corp), 'rb') as f:
        wdlist = pickle.load(f)
    locrts = Counter(get_roots('-loc', wdlist))
    keys = list(locrts.keys())
    locrts = pd.DataFrame({'morph': list(map(lambda x: x[0], keys)), 'gloss': list(map(lambda x: x[1], keys)), 
                           'pos': list(map(lambda x: x[2], keys)), 'count': list(map(lambda x: locrts[x], keys))})
    locrts = locrts.sort_values('count', ascending=False)
    locrts.to_pickle('{}_locative_roots.pickle'.format(corp))
    
s_loc = pd.read_pickle('Sebjan_locative_roots.pickle')
k_loc = pd.read_pickle('Kamchatka_locative_roots.pickle')
loc = pd.merge(s_loc, k_loc, how='outer', on=['morph', 'gloss', 'pos'], suffixes=['_s', '_k'])
loc.head()

,count_s,gloss,morph,pos,count_k
0,323.0,dist,tarak,pron,215.0
1,300.0,prox,erek,pron,88.0
2,136.0,herd.R,stado,n,NaN
3,99.0,what,ịak,pron,11.0
4,82.0,which,irek,pron,69.0


In [195]:
cases = pd.merge(cases, loc, how='outer', on=['morph', 'gloss', 'pos'], suffixes=['', '_loc'])
cases.head()

,morph,gloss,pos,count_s_dat,count_k_dat,count_s_all,count_k_all,count_s,count_k
0,tarak,dist,pron,119.0,104.0,36.0,25.0,323.0,215.0
1,ịak,what,pron,90.0,51.0,4.0,NaN,99.0,11.0
2,ireli,summer,n,76.0,1.0,NaN,NaN,NaN,NaN
3,erek,prox,pron,40.0,30.0,32.0,11.0,300.0,88.0
4,anŋan,year,n,32.0,NaN,NaN,NaN,8.0,3.0


In [196]:
cases['count_s_loc'] = cases['count_s']
cases['count_k_loc'] = cases['count_k']
del cases['count_s']
del cases['count_k']
cases.head()

,morph,gloss,pos,count_s_dat,count_k_dat,count_s_all,count_k_all,count_s_loc,count_k_loc
0,tarak,dist,pron,119.0,104.0,36.0,25.0,323.0,215.0
1,ịak,what,pron,90.0,51.0,4.0,NaN,99.0,11.0
2,ireli,summer,n,76.0,1.0,NaN,NaN,NaN,NaN
3,erek,prox,pron,40.0,30.0,32.0,11.0,300.0,88.0
4,anŋan,year,n,32.0,NaN,NaN,NaN,8.0,3.0


In [198]:
# инфа про корни
def get_roots(wdlist):
    res = []
    for wd in wdlist:
        res.append((wd[0][0], wd[1][0]))
    return res

def count_verbs(corp):
    with open('../{}_wordlist.pickle'.format(corp), 'rb') as f:
        wdlist = pickle.load(f)
    verbs = get_roots(wdlist)
    pdfr = handle_verbs(verbs)
    return pdfr

rts = pd.merge(count_verbs('Sebjan'), count_verbs('Kamchatka'), on=['morph', 'gloss'], how='outer', suffixes=['_s', '_k'])
cases = pd.merge(cases, rts, how='inner', on=['morph', 'gloss'])
cases.head(20)

,morph,gloss,pos,count_s_dat,count_k_dat,count_s_all,count_k_all,count_s_loc,count_k_loc,count_s,count_k
0,tarak,dist,pron,119.0,104.0,36.0,25.0,323.0,215.0,2789.0,1320.0
1,tarak,dist,n,NaN,2.0,NaN,NaN,1.0,NaN,2789.0,1320.0
2,ịak,what,pron,90.0,51.0,4.0,NaN,99.0,11.0,1647.0,437.0
3,ịak,what,n,NaN,1.0,NaN,NaN,1.0,NaN,1647.0,437.0
4,ireli,summer,n,76.0,1.0,NaN,NaN,NaN,NaN,112.0,3.0
5,erek,prox,pron,40.0,30.0,32.0,11.0,300.0,88.0,1741.0,718.0
6,erek,prox,n,NaN,NaN,NaN,NaN,2.0,NaN,1741.0,718.0
7,anŋan,year,n,32.0,NaN,NaN,NaN,8.0,3.0,147.0,43.0
8,mut,1pl.in,pron,31.0,NaN,4.0,1.0,46.0,2.0,559.0,29.0
9,stado,herd.R,n,31.0,NaN,6.0,NaN,136.0,NaN,263.0,NaN


In [199]:
cases.to_excel('dative_allative.xlsx')

In [194]:
cases = pd.read_excel('dative_allative.xlsx')

## Samples

In [190]:
import random


def get_all_ex_txt(corp, gloss):
    folder = '../Corpus_Text_{}_postagged'.format(corp)
    res = []
    for fil in os.listdir(folder):
        with open(os.path.join(folder, fil)) as f:
            text = f.read()
        sents = text.split('\\id')[1:]
        needed = [(fil, x) for x in sents if gloss in x]
        res += needed
    with open('text_sents_in_pickle/{}_{}.pickle'.format(gloss.strip(), corp), 'wb') as f:
        pickle.dump(res, f)

def make_samples(corp, gloss):
    with open('text_sents_in_pickle/{}_{}.pickle'.format(gloss.strip(), corp), 'rb') as f:
        alle = pickle.load(f)
    random.shuffle(alle)
    sample = ['\n'.join(x) for x in alle[:1000]]
    if not os.path.exists('samples/'):
        os.mkdir('samples/')
    with open('{}_{}'.format(gloss.strip(), corp), 'w') as f:
        f.write('\n\n'.join(sample))
        
# for corp in corpora:
#     get_all_ex_txt(corp, '-dat')
#     make_samples(corp, '-dat')

In [170]:
for corp in corpora:
    get_all_ex_txt(corp, '-all')
    make_samples(corp, '-all')
    
for corp in corpora:
    get_all_ex_txt(corp, '-loc')
    make_samples(corp, '-loc')

## Verbs

In [179]:
def detect_v(sent, gloss):
    # вытаскиваю ближайший к нужному падежу глагол, преимущество справа
    current = -1
    take = False
    res = []
    for i in range(len(sent['ps'])):
        if gloss in sent['ge'][i]:
            take = True
        elif sent['ps'][i] == ['v']:
            if take:
                res = (sent['mb'][i][0], sent['ge'][i][0])
            else:
                current = i
    if not res:
        if current > -1:
            res = (sent['mb'][current][0], sent['ge'][current][0])
        else:
            res = ('None', 'None')
    return res


def get_verbs(corp, gloss):
    print(gloss)
    verbs = []
    for doc in corp:
        for i in range(len(corp[doc]['text'])): # здесь перебор предложений
            if 'ge' in corp[doc]['text'][i] and 'ps' in corp[doc]['text'][i]: # проверяем что то что нужно
                for j in range(len(corp[doc]['text'][i]['ge'])): # перебор слов
                    try:
                        if gloss in corp[doc]['text'][i]['ge'][j]:
                            verbs.append(detect_v(corp[doc]['text'][i], gloss))
                    except:
                        pprint(corp[doc]['text'][i])
    print(len(verbs))
    return verbs


from collections import Counter


def handle_verbs(verbs):
    '''makes a pd and returns it'''
    count = Counter(verbs)
    keys = list(count.keys())
    res = pd.DataFrame({'morph': list(map(lambda x: x[0], keys)), 'gloss': list(map(lambda x: x[1], keys)), 
                        'count': [count[x] for x in keys]})
    return res


def make_verbs_table(corp, gloss):
    with open('../{}.pickle'.format(corp), 'rb') as f:
        content = pickle.load(f)
    verbs = get_verbs(content, gloss)
    pdfr = handle_verbs(verbs)
    if not os.path.exists('verbs/'):
        os.mkdir('verbs/')
    with open(os.path.join('verbs', '{}_{}.pickle'.format(gloss, corp)), 'wb') as f:
        pickle.dump(pdfr, f)

In [180]:
for corp in corpora:
    make_verbs_table(corp, '-dat')
    make_verbs_table(corp, '-all')
    make_verbs_table(corp, '-loc')

-dat
570
-all
481
-loc
1066
-dat
1213
-all
206
-loc
2219


In [197]:
# content['Afanasij_Afanasevich_making_sled_runners.txt']['text'][2]

In [133]:
pds = [x for x in os.listdir('verbs')if x.endswith('.pickle')]
together = pd.read_pickle(os.path.join('verbs', pds[0]))
together['count_all'] = together['count']
del together['count']
for i in range(1, len(pds)):
    suff = ['', '']
    second = pd.read_pickle(os.path.join('verbs', pds[i]))
    if i%2==1:
        suff = ['_k', '_s']
    if i==1:
        second['count_all'] = second['count']
        del second['count']
    elif 1<i<4:
        second['count_dat'] = second['count']
        del second['count']
    elif i>3:
        second['count_loc'] = second['count']
        del second['count']
    together = pd.merge(together, second, how='outer', on=['morph', 'gloss'], suffixes=suff)
    

['-all_Kamchatka.pickle', '-all_Sebjan.pickle', '-dat_Kamchatka.pickle', '-dat_Sebjan.pickle', '-loc_Kamchatka.pickle', '-loc_Sebjan.pickle']


In [145]:
together.head()

,gloss,morph,count_all_k,count_all_s,count_dat_k,count_dat_s,count_loc_k,count_loc_s,LL_all,LL_dat,LL_loc,count_s,count_k
0,say,goːn,12.0,34.0,13.0,122.0,35.0,260.0,3.439224,59.757065,108.506889,1488.0,954.0
2,go,hor,102.0,27.0,24.0,37.0,27.0,131.0,116.471107,1.147660,19.853362,596.0,291.0
25,sew,haŋan,NaN,NaN,4.0,6.0,2.0,13.0,0.000000,1.375134,13.203242,59.0,83.0
3,find,bak,5.0,1.0,4.0,3.0,19.0,7.0,4.746266,0.820039,11.576681,82.0,55.0
27,play,ewiː,NaN,NaN,6.0,2.0,1.0,9.0,0.000000,1.554383,8.575389,32.0,37.0


In [143]:
together.to_excel('verbs/together.xlsx')

In [111]:
from scripts import loglike

In [110]:
del together['LL_all']
del together['LL_dat']
del together['LL_loc']

In [146]:
together = loglike(together, together['count_k'], together['count_s'], columns=['count_all_k', 'count_all_s'])
together['LL_all'] = together['LL']
together = loglike(together, together['count_k'], together['count_s'], columns=['count_dat_k', 'count_dat_s'])
together['LL_dat'] = together['LL']
together = loglike(together, together['count_k'], together['count_s'], columns=['count_loc_k', 'count_loc_s'])
together['LL_loc'] = together['LL']
del together['LL']
together.head()

,gloss,morph,count_all_k,count_all_s,count_dat_k,count_dat_s,count_loc_k,count_loc_s,LL_all,LL_dat,LL_loc,count_s,count_k
0,say,goːn,12.0,34.0,13.0,122.0,35.0,260.0,3.439224,59.757065,108.506889,1488.0,954.0
2,go,hor,102.0,27.0,24.0,37.0,27.0,131.0,116.471107,1.147660,19.853362,596.0,291.0
25,sew,haŋan,NaN,NaN,4.0,6.0,2.0,13.0,0.000000,1.375134,13.203242,59.0,83.0
3,find,bak,5.0,1.0,4.0,3.0,19.0,7.0,4.746266,0.820039,11.576681,82.0,55.0
27,play,ewiː,NaN,NaN,6.0,2.0,1.0,9.0,0.000000,1.554383,8.575389,32.0,37.0


А теперь вытащу инфу про глаголы

In [139]:
def get_roots(tag, wdlist):
    res = []
    for wd in wdlist:
        if tag in wd[2]:
            res.append((wd[0][0], wd[1][0]))
    return res

def count_verbs(corp):
    with open('../{}_wordlist.pickle'.format(corp), 'rb') as f:
        wdlist = pickle.load(f)
    verbs = get_roots('v', wdlist)
    pdfr = handle_verbs(verbs)
    return pdfr

vbs = pd.merge(count_verbs('Sebjan'), count_verbs('Kamchatka'), on=['morph', 'gloss'], how='outer', suffixes=['_s', '_k'])
together = pd.merge(together, vbs, how='inner', on=['morph', 'gloss'])
together.head(20)

,gloss,morph,count_all_k,count_all_s,count_dat_k,count_dat_s,count_loc_k,count_loc_s,LL_all,LL_dat,LL_loc,count_s,count_k
0,say,goːn,12.0,34.0,13.0,122.0,35.0,260.0,37.653628,38.085424,67.896289,1488.0,954.0
1,look.at,kojeːt,8.0,4.0,3.0,1.0,19.0,6.0,0.062797,3.114218,19.920950,61.0,149.0
2,go,hor,102.0,27.0,24.0,37.0,27.0,131.0,5.398056,1.472731,19.056561,596.0,291.0
3,find,bak,5.0,1.0,4.0,3.0,19.0,7.0,0.566852,1.873821,17.969970,82.0,55.0
4,sit.down,teg,1.0,NaN,9.0,24.0,6.0,53.0,0.000000,0.344254,16.292939,177.0,57.0
5,speak,tore,NaN,1.0,2.0,5.0,10.0,2.0,0.000000,0.037898,13.264838,66.0,131.0
6,put.on,tet,2.0,NaN,8.0,4.0,9.0,2.0,0.000000,6.051926,11.396335,48.0,58.0
7,work,gurgeːwči,5.0,NaN,29.0,27.0,40.0,39.0,0.000000,9.383941,11.131851,101.0,131.0
8,set,tul,NaN,NaN,3.0,11.0,7.0,1.0,0.000000,0.768678,10.512644,19.0,21.0
9,migrate,nulge,8.0,6.0,10.0,15.0,2.0,25.0,1.035543,0.713763,9.858503,156.0,53.0


In [144]:
speech = [('goːn', 'say'), ('tore', 'speak'), ('ukčen', 'tell')]
movement = [('hor', 'go'), ('em', 'come')]
other = ['e', 'neg']

In [161]:
def get_verb_sample(v):
    morph = ' {} '.format(v[0])
    for corp in corpora:
        get_all_ex_txt(corp, morph)
        make_samples(corp, morph)
        
for v in speech+movement+other:
    get_verb_sample(v)

In [169]:
def get_zero_txt(corp, gloss):
    gloss = ' {} '.format(gloss)
    folder = '../Corpus_Text_{}_postagged'.format(corp)
    res = []
    for fil in os.listdir(folder):
        with open(os.path.join(folder, fil)) as f:
            text = f.read()
        sents = text.split('\\id')[1:]
        needed = [(fil, x) for x in sents if gloss not in x]
        res += needed
    with open('text_sents_in_pickle/{}_{}.pickle'.format(gloss.strip(), corp), 'wb') as f:
        pickle.dump(res, f)

for corp in corpora:
    get_zero_txt(corp, 'v')
    make_samples(corp, 'v')

In [200]:
with open('samples/n_Kamchatka.pickle', 'rb') as f:
    content = pickle.load(f)
content[:5]

FileNotFoundError: [Errno 2] No such file or directory: 'samples/n_Kamchatka.pickle'